### 2D kernel density
Measure 2D kernel density of points, and select points in dense region. 

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

def kernel_density(xx, yy, xmin=None, xmax=None,
                   ymin=None, ymax=None, xpix=100, ypix=100):
    import numpy as np
    import scipy.stats as st
    if xmin is None: xmin = min(xx)
    if xmax is None: xmax = max(xx)
    if ymin is None: ymin = min(yy)
    if ymax is None: ymax = max(yy)

    xpos, ypos = np.mgrid[xmin:xmax:xpix * 1j, ymin:ymax:ypix*1j]
    positions = np.vstack([xpos.ravel(), ypos.ravel()])
    values = np.vstack([xx, yy])
    kernel = st.gaussian_kde(values)
    f = np.reshape(kernel(positions).T, xpos.shape)
    return xx, yy, f

In [18]:
# Roughly, G*M = R*sig^2. (virialized)
import pickle
mass, sig3d, reff = pickle.load(open("mass_sig_r.pickle", 'rb'))


G = 6.67408e-11 #[m3 kg-1 s-2]

sig = sig3d * 1e3 # [m/s]
msun = 1.989e30 #[kg]
mass = mass * msun
kpc_to_m = 3.0857e16 #[m]
reff = reff * kpc_to_m

xx = G * mass
yy = sig**2 * reff

log = True
clip = True

if log:
    xx = np.log10(xx)
    yy = np.log10(yy)

if clip:
    xmin = min(xx)
    xmax = max(xx)
    ymin = min(yy)
    ymax = max(yy)
    
    xpos, ypos, f = kernel_density(xx, yy, 
                                   xmin=xmin, ymin=ymin, 
                                   xmax=xmax, ymax=ymax,
                                   xpix = 100, ypix=100)
    
    # if the original point belongs to the pixel with f higher than the threshold
    # it is a valid point.
    x_ngp = np.rint((xx - xmin)/(xmax - xmin) * 100 + 0.5).astype(int)
    x_ngp[x_ngp == 100] = 99
    y_ngp = np.rint((yy - ymin)/(ymax - ymin) * 100 + 0.5).astype(int)
    y_ngp[y_ngp == 100] = 99
    
    ok = f[x_ngp, y_ngp] > 0.1 * f.max()

fig, ax = plt.subplots()
ax.scatter(xx, yy)
# Color high density points
ax.scatter(xx[ok], yy[ok], color='r')
slope, intercept, r_value, p_value, std_err = stats.linregress(xx[ok],yy[ok])
xpoints = np.linspace(min(xx), max(xx), 10)
ax.plot(xpoints, slope * xpoints + intercept, 'r--')

plt.show()
